In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):") or "1"
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file: SigSpatial_data/Canyon_Lake_Gorge_TX.off



********************
tin_directory:  SigSpatial_data


Is the data stored in hdfs(0) or SigSpatial(1): 1


tin_basename:  Canyon_Lake_Gorge_TX.off
tin_filename:  Canyon_Lake_Gorge_TX
tin_extension:  .off

********************


In [4]:
# filtra is the order of each vertex, the order is obtained by ranking elevation values of vertices
filtra = input("Do you have filtration data?") or "yes"
# filtra = 'yes'

if filtra.lower() == 'no':    
    Basic_Data = input("Do you have basic pts and tri data?")
    
Num_executor = input("spark.executor.instances:") or "64"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "64g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "8g"

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor

Num_core_per_driver = '5'
Memory_driver = '12g'
MemoryOverhead_driver = '4g'

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "200"

Do you have filtration data? yes
spark.executor.instances: 64
spark.executor.cores: 5
spark.executor.memory? Please end with 'g': 64g
spark.executor.memoryOverhead? Please end with 'g': 8g
spark.sql.shuffle.partitions: 200


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

# set the Spark app name
spark_app_name = "Forman_final_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '200') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: Forman_final_Canyon_Lake_Gorge_TX_07112024_1340


### read input files

In [8]:
# read_ver_order() is a function used to read vertices from a csv file
def read_ver_order(filtra, directory, tin_filename):
    '''
    this function has three input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a TIN without extension, e.g., Canyon_Lake_Gorge_TX
    '''
    if filtra.lower() == 'yes': # 'yes' means we have already computed the filtra value for each vertex
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True), \
            StructField("self_order",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_filtra_pts_origin.csv'
    
        df_ver_order = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    if filtra.lower() == 'no': # 'no' means we need to rank vertices to get their filtra values
        schema_ver_origin = StructType([ \
            StructField("x",FloatType(),True), \
            StructField("y",FloatType(),True), \
            StructField("ele",FloatType(),True), \
            StructField("self_index",IntegerType(),True) \
          ])
        
        hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'
    
        df_ver_origin = spark.read.format("csv") \
              .option("header", False) \
              .schema(schema_ver_origin)\
              .load(hdfs_tin_pts_origin)
        
        print("Number of partitions for df_ver_origin:", df_ver_origin.rdd.getNumPartitions())
        
        # define a window for the ordering
        # row_number() function along with partitionBy() of other column populates the row number by group
        # Since we want to order the whole DataFrame, so we don't need the partitionBy() function
        w = Window().orderBy(col('ele').asc())
        df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
        print("Number of partitions for df_ver_order:", df_ver_order.rdd.getNumPartitions())
        
    return df_ver_order

# read vertices
df_ver_order = read_ver_order(filtra, directory, tin_filename)
df_ver_order.printSchema()

Number of partitions for df_ver_order: 200
root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)



In [9]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tri_order(hdfs_tin_tri_origin):
    '''
    hdfs_tin_tri_origin: path to the input triangle csv file
    '''
        
    schema_tri_origin = StructType([ \
        StructField("tri_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True)
      ])

    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
        
    return df_tri_origin


# read tetrahedra
hdfs_tin_tri_origin = directory + "/" + tin_filename + '_filtra_tri_origin.csv'

df_tri_order = read_tri_order(hdfs_tin_tri_origin)
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



##### add a column for representing a triangle as an array of vertices

In [10]:
# sort the extreme vertices of a triangle in ascending order, e.g., (2,5,3) -> (2,3,5)

'''
def get_tri_array(r1, r2, r3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
    tri = [r1, r2, r3]    
    tri.sort(reverse=True)
    
    return tri

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_tri_array_udf = udf(get_tri_array, ArrayType(IntegerType()))
df_tri_order = df_tri_order.withColumn("tri", get_tri_array_udf(df_tri_order.r1, df_tri_order.r2, df_tri_order.r3))
'''

df_tri_order = df_tri_order.withColumn("tri_origin", F.array("r1", "r2", "r3")).withColumn("tri_ele_origin", F.array("r1_ele", "r2_ele", "r3_ele"))
df_tri_order = df_tri_order.withColumn("tri", sort_array("tri_origin", False)).drop("tri_origin") # sort_array("tri", False), False means descending order
df_tri_order = df_tri_order.withColumn("tri_ele", sort_array("tri_ele_origin", False)).drop("tri_ele_origin") # sort_array("tri", False), False means descending order

df_tri_order.printSchema()
df_tri_order.rdd.getNumPartitions()
#df_tri_order.show()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- tri: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- tri_ele: array (nullable = false)
 |    |-- element: float (containsNull = true)



200

### globally get VT relation

In [11]:
# function to get VT directly from DF_T
def get_VT(df_tri_order):
    
    df_VT_init_1 = df_tri_order.select("r1","tri")
    df_VT_init_2 = df_tri_order.select("r2","tri")
    df_VT_init_3 = df_tri_order.select("r3","tri")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    
    df_VT = df_VT_union123.groupBy('r1').agg(collect_list('tri').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [12]:
df_VT = get_VT(df_tri_order)
# df_VT = df_VT.withColumn('VT', df_VT.VT.cast(ArrayType(ArrayType(StringType()))))
# df_VT = df_VT.withColumn('VT', df_VT.VT.cast(StringType()))

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [16]:
t0 = time.time()

num_df_VT = df_VT.count()
print("number of rows in df_VT:", num_df_VT)

t1 = time.time()
print("time for computing df_VT:", t1 - t0)

number of rows in df_VT: 97977344
time for computing df_VT: 5.684948921203613


### locally get VV relation

In [13]:
# function to get VV from VT
def get_VV_from_VT(Ver, VT):
    pt_set = set()
    if len(VT) > 0:
        for tri in VT:
            pt_set.update(tri)
    
    pt_set.remove(Ver)
    VV = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return VV # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VV_from_VT_udf = udf(get_VV_from_VT, ArrayType(IntegerType()))

In [14]:
df_VV = df_VT.withColumn("VV", get_VV_from_VT_udf(df_VT.Ver, df_VT.VT)).drop('VT')

df_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [17]:
t0 = time.time()

num_df_VV = df_VV.count()
print("number of rows in df_VV:", num_df_VV)

t1 = time.time()
print("time for computing df_VV:", t1 - t0)

number of rows in df_VV: 97977344
time for computing df_VV: 5.716812372207642


### locally get VE relation

In [18]:
def get_VE_from_VV(Ver, VV):
    # df_VT has three columns, Ver, Ver_ele, and VT_filtra
    
    if VV:
        multi_E = []
        for v_adj in VV:
            if Ver > v_adj:
                multi_E.append([Ver, v_adj])
            else:
                multi_E.append([v_adj, Ver])
        return multi_E
             
get_VE_from_VV_udf = udf(get_VE_from_VV, ArrayType(ArrayType(IntegerType())))

In [19]:
df_VE = df_VV.withColumn("VE", get_VE_from_VV_udf(df_VV.Ver, df_VV.VV))
df_VE = df_VE.select("Ver", "VE")
df_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [20]:
t0 = time.time()

num_df_VE = df_VE.count()
print("number of rows in df_VE:", num_df_VE)

t1 = time.time()
print("time for computing df_VE:", t1 - t0)

number of rows in df_VE: 97977344
time for computing df_VE: 5.298813581466675
